In [12]:
import polars as pl
from sqlalchemy.engine import create_engine
import unicodedata
import requests
import zipfile
import io

In [13]:
url = "https://portaldatransparencia.gov.br/download-de-dados/despesas-favorecidos/{year}{month}".format(
    year="2024", month="05"
)

In [14]:
response = requests.get(url, stream=True)
zip_data = zipfile.ZipFile(io.BytesIO(response.content))
csv = zip_data.namelist()[0]
file = zip_data.open(csv).read()

In [15]:
df = pl.read_csv(
    file,
    separator=";",
    encoding="1252",
    decimal_comma=True,
)

df.columns = [
    unicodedata.normalize("NFD", col.lower().replace(" ", "_"))
    .encode("ascii", "ignore")
    .decode("utf-8")
    for col in df.columns
]

df = df.with_columns(pl.col("ano_e_mes_do_lancamento").str.strptime(pl.Date, "%m/%Y"))

In [16]:
# df.select(pl.col(["nome_favorecido", "nome_municipio", "valor_recebido"])).sort(by=pl.col("valor_recebido"), descending=True).head(10)
df.schema

OrderedDict([('codigo_favorecido', String),
             ('nome_favorecido', String),
             ('sigla_uf', String),
             ('nome_municipio', String),
             ('codigo_orgao_superior', Int64),
             ('nome_orgao_superior', String),
             ('codigo_orgao', Int64),
             ('nome_orgao', String),
             ('codigo_unidade_gestora', Int64),
             ('nome_unidade_gestora', String),
             ('ano_e_mes_do_lancamento', Date),
             ('valor_recebido', Float64)])